<img src="Archivos/miad4.png" width=800x>

# Modificar bases de datos  

Al finalizar la exploración de una base de datos procedemos a modificarla. En este tutorial expandimos la explicación de los métodos de la librería `pandas` vistos anteriormente y presentamos métodos adicionales para integrarlos en procesos de modificación de bases de datos. 

## Requisitos

Para desarrollar este tutorial necesitarás:

* Importar archivos de texto.
* Implementar filtrado condicional y filtrado numérico.
* Utilizar los métodos básicos de los `Series` y los `DataFrame`.

## Objetivos

Al final de este tutorial podrás:

**1.** Procesar archivos en formatos especiales. <br>
**2.** Modificar la estructura y el contenido de un `DataFrame`.

## 1. Procesamiento de archivos en formatos especiales 

La librería `pandas` nos permite importar archivos en múltiples formatos para procesarlos como un `DataFrame`. Entre estos formatos, se encuentran los archivos de tipo texto, de tipo binario y de tipo SQL. En las referencias encuentras la lista completa de métodos de lectura y escritura según el tipo de archivo. A manera de ejemplo expondremos el método `read_fwf` el cuál permite procesar archivos de ancho fijo. 

    pd.read_fwf(filepath_or_buffer, colspecs, **kwds)

* **filepath_or_buffer:** ruta del archivo de texto a procesar.  

* **colspecs:** lista de tuplas que identifica la posición donde empieza y termina cada columna. Especificando `colspecs = 'infer'` el método infiere las columnas del archivo. 

* ** **kwds:** indica que podemos especificar los parámetros disponibles en el método `read_csv`. 

##### Ejemplo 1 

Retomemos el archivo `"mfh.txt"` (*Major foreign holders*) el cual contiene la distribución del monto de títulos del tesoro americano de los Estados Unidos que se encuentra en cuentas afuera de su país. Dado que este es un archivo de ancho fijo, utilizaremos parte del código que aprendimos anteriormente para obtener las posiciones de inicio y de fin de las columnas que conforman el archivo.

In [ ]:
with open("Archivos/mfh.txt", "r") as mfh: 
    # Leemos el archivo y lo almacenamos en una sola cadena de caracteres
    mfh = mfh.read()
    # Seleccionamos la cadena de caracteres que se encuentra entre la palabra 'Period' y la palabra 'Grand'
    mfh = mfh.split('PERIOD')[1].split('Grand')[0]
    # Declaramos una lista con las líneas de nuestra cadena de caracteres
    lista_mfh = mfh.splitlines()
    # Nos quedamos únicamente con las líneas no vacías
    lista_mfh = [x for x in lista_mfh if x]

Declaramos las siguientes variables: 

In [ ]:
# Lista vacía para almacenar las posiciones de inicio de cada columna de números.
lista_inicio_cols = []
# Lista vacía para almacenar la última posición de cada columna de números. 
lista_fin_cols = []
# Cadena de caracteres que almacena la fila de guíones contenida en el archivo.
fila_guiones = lista_mfh[2]
# Variable de tipo entero para marcar el inicio de la busqueda a partir de cada columna. 
inicio_columna = 0
# Variable de tipo lógica para indicar si ya llegamos al final de la lista.
criterio_parada = True

Iteramos mientras el booleano se mantenga igual a `True`. 

In [ ]:
while(criterio_parada):
    # Declaramos el inicio de una nueva columna encontrando la palabra guión.
    pos_inicio = inicio_columna + fila_guiones[inicio_columna:].find('-')
    # Declaramos el ancho de la columna encontrando el siguiente espacio vacío.  
    ancho_columna = fila_guiones[pos_inicio:].find(' ')
    # Si encuentra dicho espacio
    if(ancho_columna != -1):
        # Almacenamos la posición de inicio de la columna.
        lista_inicio_cols.append(pos_inicio)
        # Declaramos la posición final de la columna.
        pos_final = pos_inicio + ancho_columna 
        # Almacenamos la posición final de la columna.
        lista_fin_cols.append(pos_final) 
        # Actualizamos el inicio de la columna para la siguiente iteración.
        inicio_columna = pos_final
    #Si no se pudo encontrar mas espacios no se se sigue iterando ya que no hay mas columnas.    
    else:
        criterio_parada = False

El código anterior nos permite obtener las posiciones en las que empiezan y finalizan las columnas de números. Entonces agreguemos la tupla que contiene la posición donde empieza la columna de países (posición 0) y la posición donde empieza la primera columna de números.

In [ ]:
ancho_cols = [(0,lista_inicio_cols[0])]+list(zip(lista_inicio_cols, lista_fin_cols))
ancho_cols

Importemos las librerías necesarias.

In [ ]:
import numpy as np 
import pandas as pd

Luego, utilizamos el método `read_fwf` para leer el archivo, omitiendo las filas correspondientes al encabezado y al pie de tabla. Además, para efectos de este tutorial debemos omitir los registros correspondientes al gran recuento total ('Grand Total') y los demás países ('All other').

In [ ]:
mfh = pd.read_fwf("Archivos/mfh.txt", skiprows = range(8), skipfooter = 21,
                  colspecs = ancho_cols, index_col = [0], header = [0,1])
# Eliminamos las dos primeras entradas ya que no contienen información relevante. 
mfh = mfh[1:-2]
mfh

Al observar el `DataFrame` de la celda anterior nos damos cuenta que sería deseable rotular correctamente los niveles de las columnas y el índice, para esto podemos usar el método `rename_axis`: 

In [ ]:
mfh = mfh.rename_axis("Pais")
mfh = mfh.rename_axis(["Mes","Año"], axis = 1)
mfh

## 2. Métodos para la modificación de bases de datos  

### 2.1. Método `set_index` 

Permite utilizar una o varias columnas para sustituir o modificar el índice de un `DataFrame`. Además de columnas podemos utilizar arreglos de `pandas` (`Series` e `Index`) o de arreglos de `numpy`. 

    pandas.DataFrame.set_index(keys, drop, append, inplace, verify_integrity)

* **keys:** indica la(s) llave(s) a usar como nuevo índice.

* **drop:** elimina del `DataFrame` las columnas que se emplearon como llaves en el método. Por defecto es `True`.

* **append:** preseva el índice original de la base de datos y le agrega la llave ingresada por parámetro. Por defecto es `False`.


* **inplace:** por defecto es `False`. <br><br>
    * `inplace = True`: retorna `None` y  ejecuta las operaciones sobre el dataframe oríginal. <br>
    * `inplace = False`: retorna una copia del `DataFrame` con las modificaciones. <br><br>
    
* **verify_integrity:** solo aplica si ignore_index = False. Por defecto, `verify_integrity = False`.<br><br>
    * `verify_integrity = True`: arroja ValueError si en el DataFrame resultante hay indices duplicados.<br>
    * `verify_integrity = False`: permite tener índices repetidos en el DataFrame resultante.<br><br>

**Nota:** si utilizamos un arreglo externo al dataframe, debemos asegurarnos que tenga el mismo tamaño que el que tiene actualmente el `DataFrame`.

##### Ejemplo 2

Requerimos analizar como se distribuye la tenencia de los títulos del tesoros entre dos grupos de países: los países miembros de la OECD y los países llamados paraísos fiscales. Ambos grupos no son mutuamente excluyentes por lo que pueden haber coincidencias entre ambos. A continuación, se encuentra declarada una lista para cada grupo de países.

In [ ]:
lista_paraisos_fiscales = ["Belgium", "Ireland", "Luxembourg", "Malta", "Netherlands", "Cayman Island",
                           "Singapore", "Puerto Rico", "Hong Kong", "Switzerland"]

lista_oecd = ["Austria", "Australia", "Belgium", "Canada", "Chile", "Colombia", "Czech Republic", "Denmark", 
              "Estonia", "Finland", "France", "Germany", "Greece", "Hungary", "Iceland", "Ireland", "Israel", 
              "Italy", "Japan", "Korea", "Latvia", "Lithuania", "Luxembourg", "Mexico",  "Netherlands", 
              "New Zealand", "Norway", "Poland", "Portugal", "Slovak Republic", "Slovenia", "Spain", "Sweden", 
              "Switzerland", "Turkey", "United Kingdom", "United States"]

Creamos las columnas correspondientes para posteriormente añadirlas al índice.

In [ ]:
# Creamos las columnas "Clasificación Fiscal" y "OECD" 
mfh.loc[:,'Clasificación_Fiscal'] = "Tributación Regular"
mfh.loc[:,'OECD'] = "No pertenece"
# Definimos cuales países son considerado paraisos fiscales y cuáles son miembros de la OECD
mfh.loc[mfh.index.isin(lista_paraisos_fiscales), 'Clasificación_Fiscal'] = "Paraiso Fiscal"
mfh.loc[mfh.index.isin(lista_oecd), 'OECD'] = "Pertenece"
# Utilizamos set_index para añadir estas columnas al índice
mfh.set_index(keys =['Clasificación_Fiscal','OECD'], append = True, inplace = True)
mfh

### 2.2. Método `reset_index`

Resetea el índice de un `DataFrame`. Si se trata de un multi-índice permite eliminar uno o más niveles, dejando la opción de preservarlos como columnas.

    pandas.DataFrame.reset_index(level, drop)

* **level:** etiqueta o índice del multi-índice a eliminar. <br><br>

* **drop:** por defecto es `False` <br><br>
`Drop = True`: se eliminan los niveles. <br><br>
`Drop = False`: los niveles pasan a ser una columna. 

### 2.3. Método `drop`

Permite eliminar filas o columnas especificando su nombre o su índice. 

    pandas.DataFrame.drop(labels, axis, index, columns, level, inplace, errors)

* **labels:** indica el nombre o la posición de las filas o columnas a eliminar.

* **axis:** por defecto es 0. <br><br>
    * `axis = 1`: indica que vamos a eliminar columnas. <br>
    * `axis = 0`: indica que vamos a eliminar filas.<br><br>


* **index:** se utiliza únicamente para eliminar filas, especificando el nombre o el índice de las filas a eliminar. Cuando utilizamos este parámetro no es necesario utilizar los parámetros `labels` y `axis`.

* **columns:** se utiliza únicamente para eliminar columnas, especificando el nombre o el índice de las columnas a eliminar. Cuando utilizamos este parámetro no es necesario utilizar los parámetros `labels` y `axis`.

* **level:** indica por nombre o por posición a cuál de los niveles del índice múltiple se le aplicará el método.


* **inplace:** por defecto es `False`. <br><br>
    * `inplace = True`: retorna `None` y  ejecuta las operaciones sobre el dataframe oríginal. <br>
    * `inplace = False`: retorna una copia del `DataFrame` con las modificaciones. <br><br>

##### Ejemplo 3

Requerimos eliminar las observaciones del `DataFrame` `mfh` correspondientes al año 2020, y almacenar el `DataFrame` resultante en un `DataFrame` llamado `mfh_2021`.

Procedemos entonces utilizando el método `drop`.

In [ ]:
mfh_2021 = mfh.drop(labels = '2020', axis = 1, level = 1 )  
mfh_2021

### 2.4. Método `Groupby`

El método groupby permite realizar las siguientes tres operaciones a la vez: 
* (1) Separar en grupos el `DataFrame` de acuerdo a un criterio.
* (2) Aplicar una función a cada grupo. 
* (3) Combinar los resultados en un nuevo `DataFrame`.

La sintaxis para usar groupby es la siguiente:

    pandas.DataFrame.groupby(by, axis, level)

* **by:** indica el criterio para realizar el paso (1). Puede usarse un diccionario, un `Series`, entre otros.

* **axis:** por defecto es 0. <br><br>
    * `axis = 1`: indica que vamos a agrupar columnas. <br>
    * `axis = 0`: indica que vamos a agrupar filas.<br><br>

* **level:** indica por nombre o por posición a cuál de los niveles del índice múltiple se le aplicará el método.

##### Ejemplo 4

Requerimos un `DataFrame` llamado `mfh_resumen` que sume el total de títulos del tesoro para los niveles del índice múltiple. 

Transformamos los valores de nuestro `DataFrame` a `float` y después aplicamos el método `groupby`.

In [ ]:
mfh = mfh.astype(float)
mfh_resumen = mfh.groupby(level = ['Clasificación_Fiscal','OECD']).sum()
mfh_resumen

### 2.5. Método `apply`

El método `apply` lleva a cabo una función bien sea sobre las columnas o sobre las filas de un `DataFrame`. 

    pandas.DataFrame.apply(func, axis)

* **func:** función a aplicar. <br><br>

* **axis:**  por defecto es 0. <br><br>

    * `axis = 1`: indica que vamos a aplicar la función sobre las columnas. <br>
    * `axis = 0`: indica que vamos a aplicar la función sobre las filas.<br><br>

A continuación explicaremos tres ventajas que presenta el método `apply`: 

#### Trabajar con funciones declaradas previamente 

El parámetro `func` puede tomar el valor de una función declarada previamente.

##### Ejemplo 5

Requerimos calcular el porcentaje de títulos distribuido en las cuentas de los países clasificados como paraíso fiscal y guardar este resultado en un `DataFrame` llamado `mfh_paraisos_fiscales`. 

Para esto declaramos la función `calcular_porcentaje`.

In [ ]:
def calcular_porcentaje(lista):
    
    rta = lista / sum(lista)
    
    return rta 

Después la usamos como parámetro del método `apply`.

In [ ]:
# Declaramos el Segmentador de Índices
idx = pd.IndexSlice
mfh_paraisos_fiscales = mfh.loc[idx[:,'Paraiso Fiscal',],]
mfh_paraisos_fiscales = mfh_paraisos_fiscales.apply(calcular_porcentaje)
mfh_paraisos_fiscales

#### Encadenar métodos 

En `pandas` podemos encadenar métodos, esto significa que se puede invocar un método tras otro sin redefinir el `DataFrame` repetidamente. El método `apply` puede encadenarse con otros métodos, lo que resulta práctico en términos de ahorro de líneas de código y claridad de lectura. 

##### Ejemplo 6

Filtra la base para solo trabajar con aquellos países que son parte de la OECD, de tal forma que pueda calcular el porcentaje de títulos bajo posesión de cada país.

Hacemos entonces el filtrado y directamente realizamos el `apply`.

In [ ]:
mfh_oecd = mfh.loc[idx[:,:,'Pertenece'],].apply(calcular_porcentaje,axis = 0)
mfh_oecd

#### Múltiples operaciones simultaneamente

El método `apply` permite ejecutar varias operaciones a la vez, lo cual nos ahorra líneas de código o encadenar más métodos de los necesarios. 

##### Ejemplo 7

Requerimos nuevamente filtrar la base para solo trabajar con aquellos países que son parte de la OECD, de tal forma que calculemos el porcentaje de títulos bajo posesión de cada país. Esta vez, redondearemos el resultado a tres cifras decimales. 

Una manera de solucionar este requerimiento es utilizar la división de objetos y el método `round` de `pandas`.

In [ ]:
mfh_oecd = mfh.loc[idx[:,:,'Pertenece'],]
mfh_oecd_norm = (mfh_oecd / mfh_oecd.sum()).round(3)
mfh_oecd_norm

En cambio podríamos haber usado una sola línea de código para cubrir este requerimiento.

In [ ]:
mfh_oecd_norm = mfh.loc[idx[:,:,'Pertenece'],].apply(lambda x: round(x / x.sum(),3))
mfh_oecd_norm

### 2.7. Método `rename`

El método `rename` sirve para renombrar columnas o índices. 

    pandas.DataFrame.rename(mapper, index, columns, axis, inplace, level)

* **mapper:** diccionario o función para renombrar las columnas o índices. <br><br>

* **index:** diccionario o función para renombrar índices. Cuando utilizamos este parámetro no es necesario utilizar los parámetros `mapper` y `axis`. <br><br>

* **columns:** diccionario o función para renombrar columnas. Cuando utilizamos este parámetro no es necesario utilizar los parámetros `mapper` y `axis`. <br><br>

* **axis:** por defecto es 0. <br><br>
    * `axis = 1`:  indica que vamos a renombrar columnas.<br>
    * `axis = 0`: indica que vamos a renombrar filas.<br>

* **inplace:** por defecto es `False`. <br><br>
    * `inplace = True`: retorna `None` y  ejecuta las operaciones sobre el dataframe oríginal. <br>
    * `inplace = False`: retorna una copia del `DataFrame` con las modificaciones. <br><br>

* **level:** indica por nombre o por posición a cuál de los niveles del índice múltiple se le debe aplicar el diccionario o la función. 

##### Ejemplo 8

En la siguiente celda encuentras un diccionario con los nombres de los países en inglés y en español.

In [ ]:
df_paises = pd.read_csv("Archivos/DiccionarioPaises.csv", encoding = 'utf-8-sig', sep = ";", index_col = [1])
dicc_paises = df_paises.to_dict()['País en español']

A continuación encuentras un diccionario con las abreviaturas de los meses en inglés y en español.

In [ ]:
dicc_meses = {'Jan':'ENE', 'Feb': 'FEB', 'Mar':'MAR', 'Apr':'ABR', 'May':'MAY' ,'Jun': 'JUN', 'Jul':'JUL',
                    'Aug': 'AGO', 'Sep': 'SEP', 'Oct':'OCT','Nov':'NOV', 'Dec':'DIC'}

Requerimos traducir al español los meses y países del `DataFrame` utilizando los diccionarios provistos 

In [ ]:
mfh = mfh.rename(mapper = dicc_paises)
mfh = mfh.rename(columns = dicc_meses) 
mfh

### 2.8. Método `replace`

El método `replace` usa la siguiente sintaxis para reemplazar un valor por otro.

    pandas.DataFrame.replace( to_replace, value, inplace) 

* **to_replace:** valor específico, estructura de datos. <br><br>

* **value:** valor que sustituirá el valor a reemplazar.<br><br>

* **inplace:** por defecto es `False`. <br><br>
    * `inplace = True`: retorna `None` y  ejecuta las operaciones sobre el dataframe oríginal. <br>
    * `inplace = False`: retorna una copia del `DataFrame` con las modificaciones. <br><br>

### 2.9. Método `stack`

El método `stack` transfiere uno o varios niveles de las columnas al índice. Al emplearlo, el número de niveles del índice se va a ver aumentando y el número de niveles de las columnas se va a ver disminuido. 

    pandas.DataFrame.stack(level, fill_value) 

* **level:** nombre o índice del nivel a transferir. Por defecto es -1 (el último nivel) 

* **fill_value:** valor para reemplazar los datos faltantes que se generen. 

##### Ejemplo 9

Pasa el nivel correspondiente a los meses al índice de `mfh_2021`.

Procedemos entonces usando el método `stack`.

In [ ]:
mfh_2021 = mfh_2021.stack("Mes") 
mfh_2021

### 2.10. Método `unstack`

El método `unstack` transfiere uno o varios niveles del índice a las columna. Al emplearlo, el número de niveles de las columnas se va a ver aumentando y el número de niveles del índice se va a ver disminuido. 

    pandas.DataFrame.unstack(level, fill_value) 

* **level:** nombre o índice del nivel a transferir. Por defecto es -1 (el último nivel) 

* **fill_value:** valor para reemplazar los datos faltantes que se generen. 

#### Ejemplo 10

Se te solicita revertir el cambio realizado en el ejemplo anterior.

Utilizamos el método `unstack` para revertir el cambio realizado.

In [ ]:
mfh_2021 = mfh_2021.unstack("Mes")
mfh_2021

## Referencias

Pandas (2021). Documentación sobre I/O tools (text, CSV, HDF5...). Recuperado el 24 de Abril de 2020 de: https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html 

Pandas (2020). Documentación sobre los objetos índices. Recuperado el 15 de mayo de 2020 de: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.IndexSlice.html

Pandas (2020). Documentación sobre los métodos DataFrame. Recuperado el 15 de mayo de 2020 de: https://pandas.pydata.org/docs/reference/frame.html

Departamento del Tesoro Americano (2020). Principales tenedores foráneos de títulos del tesoro [Base de Datos]. Recuperado el  15 de septiembre de 2020 de : 
https://ticdata.treasury.gov/Publish/mfh.txt.

## Créditos

**Autor(es):** Jorge Esteban Camargo Forero, Alejandro Mantilla Redondo,Diego Alejandro Cely Gómez

**Fecha Última actualización:** 19/09/2021